In [8]:
import chess
import chess.engine
import random
import numpy
from stockfish import Stockfish
import os
import torch
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


def random_board(maxD=200):
    board = chess.Board()
    depth = random.randrange(0, maxD)
    
    for _ in range(depth):
        all_moves = list(board.legal_moves)
        random_move = random.choice(all_moves)
        board.push(random_move)
        if board.is_game_over():
            break
    return board

def Da_fish(board, depth):
    # stockfish = Stockfish('stockfish/stockfish/stockfish-windows-x86-64-avx2.exe')
    # stockfish.set_depth(20)
    # stockfish.set_skill_level(20)
    # stockfish.set_fen_position(board.fen())
    # return stockfish.get_evaluation()
    with chess.engine.SimpleEngine.popen_uci('stockfish/stockfish/stockfish-windows-x86-64-avx2.exe') as sf:
        result = sf.analyse(board, chess.engine.Limit(depth=depth))
        score = result['score'].white().score()
        return score


In [9]:
squares_index = {
    'a':0,
    'b':1,
    'c':2,
    'd':3,
    'e':4,
    'f':5,
    'g':6,
    'h':7,
}

def square_to_index(square):
    letter =  chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]

def split_dims(board):
    board3d = numpy.zeros((14,8,8), dtype=numpy.int8)
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = numpy.unravel_index(square, (8, 8))
            board3d[piece - 1][7 -idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = numpy. unravel_index(square, (8, 8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1
    # add attacks and valid moves too
    # so the network knows what is being attacked
    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j= square_to_index(move.to_square)
        board3d[12][i][j] = 1
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j= square_to_index(move.to_square)
        board3d[13][i][j] = 1
    board.turn = aux
    
    return board3d

In [10]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


In [4]:
import numpy as np
def get_dataset():
    container = np.load('dataeval/chess_evals.npz')
    b, z = container['positions'], container['evaluations']
    print(len(b), len(z))
    v = np.asarray((np.tanh(3*(z/abs(z).max()))+1)/2)
    return b, v, z

x_train, y_train, z = get_dataset()

x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)

X_tensor, Y_tensor = torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)

6904805 6904805


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, conv_size, conv_depth):
        super(Net, self).__init__()
        # Adjust the in_channels of the first convolutional layer to match the input data
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=14 if i == 0 else conv_size,
                                              out_channels=conv_size,
                                              kernel_size=3,
                                              padding='same')
                                    for i in range(conv_depth)])
        self.fc1 = nn.Linear(conv_size * 8 * 8, 64)  # Adjust the size accordingly if the input volume changes
        self.fc2 = nn.Linear(64, 1)
        
    def forward(self, x):
        # Apply the convolutional layers
        for conv in self.convs:
            x = F.relu(conv(x))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Adjust `conv_size` and `conv_depth` as needed
conv_size = 64
conv_depth = 5
net = Net(conv_size, conv_depth)

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)
    
    def forward(self, x):
        residual = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += residual
        out = F.relu(out)
        return out
    
class ResNetChess(nn.Module):
    def __init__(self, conv_size, conv_depth):
        super(ResNetChess, self).__init__()
        self.initial_conv = nn.Conv2d(in_channels=14, out_channels=conv_size, kernel_size=3, padding='same')
        self.res_blocks = nn.Sequential(*[ResidualBlock(conv_size) for _ in range(conv_depth)])
        self.fc1 = nn.Linear(conv_size * 8 * 8, 64)
        self.fc2 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = F.relu(self.initial_conv(x))
        x = self.res_blocks(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

conv_size = 64
conv_depth = 5
net = ResNetChess(conv_size, conv_depth)



In [6]:
import torch.optim as optim
import torch.nn as nn
import numpy
import torch
from torch.utils.data import TensorDataset, DataLoader

# Assuming your code and the Net class definition are here

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

# Initial learning rate
lr_initial = 0.001
# Set the optimizer with the initial learning rate
optimizer = optim.Adam(net.parameters(), lr=lr_initial)
criterion = nn.BCELoss()

dataset = TensorDataset(X_tensor, Y_tensor)
dataloader = DataLoader(dataset, batch_size=512, shuffle=True)

# Training loop
num_epochs = 5  # Adjust the number of epochs as needed

# Calculate gamma for exponential decay
lr_final = 0.0005
gamma = (lr_final / lr_initial) ** (1 / num_epochs)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

for epoch in range(num_epochs):
    running_loss = 0.0
    ssss = 0
    for inputs, labels in dataloader:
        ssss += 1
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device in batches
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        outputs = torch.squeeze(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if ssss%500 == 0:
        
            print(f"loss at step {ssss}: {loss.item()*1000}")

        running_loss += loss.item()
    
    # Decay the learning rate
    scheduler.step()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)*1000}')

print('Finished Training')



loss at step 500: 692.1298503875732
loss at step 1000: 692.2876238822937
loss at step 1500: 691.4325952529907
loss at step 2000: 691.941499710083
loss at step 2500: 690.6639337539673
loss at step 3000: 690.5879974365234
loss at step 3500: 690.5093193054199
loss at step 4000: 690.9477710723877
loss at step 4500: 691.0675764083862
loss at step 5000: 692.0304298400879
loss at step 5500: 692.315936088562
loss at step 6000: 691.8141841888428
loss at step 6500: 689.4163489341736
loss at step 7000: 689.3925070762634
loss at step 7500: 689.8553371429443
loss at step 8000: 689.3571615219116
loss at step 8500: 690.1525855064392
loss at step 9000: 688.3330345153809
loss at step 9500: 690.5763149261475
loss at step 10000: 690.6561255455017
loss at step 10500: 690.8711194992065
loss at step 11000: 692.8887367248535
loss at step 11500: 690.293550491333
loss at step 12000: 688.2592439651489
loss at step 12500: 689.7434592247009
loss at step 13000: 688.8603568077087
Epoch 1, Loss: 690.9198578274181
lo

In [16]:
#torch.save(net.state_dict(), 'model_weights.pth')
net

ResNetChess(
  (initial_conv): Conv2d(14, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (res_blocks): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Bat

In [26]:
import torch.optim as optim
import torch.nn as nn
import numpy
import torch
from torch.utils.data import TensorDataset, DataLoader

# Assuming your code and the Net class definition are here

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.BCELoss()


# Create a dataset. Note: Do not move the entire dataset to the device here
dataset = TensorDataset(X_tensor, Y_tensor)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# Training loop
num_epochs = 1  # Adjust the number of epochs as needed

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device in batches
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        outputs = torch.squeeze(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')
    
torch.save(net.state_dict(), 'model_weights.pth')

print('Finished Training')

Epoch 1, Loss: 0.6926039990938755
Finished Training


In [15]:
import numpy as np
import torch.optim as optim
import torch.nn as nn
import numpy
import torch
from torch.utils.data import TensorDataset, DataLoader


def get_dataset(doc):
    container = np.load(doc)
    b, z = container['positions'], container['evaluations']
    print(len(b), len(z))
    v = np.asarray(z/abs(z).max()/2+0.5)
    return b, v, z

docs = [
    'dataeval/chess_evals.npz',
    'dataeval/random_evals.npz',
    'dataeval/tactics_evals.npz'
]

allZS = []

for doc in docs:
    x_train, y_train, z = get_dataset(doc)
    
    allZS.append(z)

    x_train = x_train.astype(np.float32)
    y_train = y_train.astype(np.float32)

    X_tensor, Y_tensor = torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)



    # Assuming your code and the Net class definition are here

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.to(device)

    optimizer = optim.Adam(net.parameters(), lr=0.001)
    criterion = nn.BCELoss()


    # Create a dataset. Note: Do not move the entire dataset to the device here
    dataset = TensorDataset(X_tensor, Y_tensor)
    dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

    # Training loop
    num_epochs = 10  # Adjust the number of epochs as needed

    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to device in batches
            
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = net(inputs)
            outputs = torch.squeeze(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')
    
# torch.save(net.state_dict(), 'model_weights.pth')
print(allZS)

print('Finished Training')

6904805 6904805
Epoch 1, Loss: 0.692553937472718
Epoch 2, Loss: 0.6925201222784492
Epoch 3, Loss: 0.6924936466065974
Epoch 4, Loss: 0.6924719352070168
Epoch 5, Loss: 0.6924518246049522
Epoch 6, Loss: 0.6924331563659013
Epoch 7, Loss: 0.6924164594976605
Epoch 8, Loss: 0.6924000959199772
Epoch 9, Loss: 0.6923837563459144
Epoch 10, Loss: 0.6923694579845346
2345781 2345781
Epoch 1, Loss: 0.6922634187765647
Epoch 2, Loss: 0.6920715767473312
Epoch 3, Loss: 0.6919812778602575
Epoch 4, Loss: 0.6919121772624885
Epoch 5, Loss: 0.6918553032142386
Epoch 6, Loss: 0.6918049996487344
Epoch 7, Loss: 0.6917563942318209
Epoch 8, Loss: 0.691711490267576
Epoch 9, Loss: 0.6916697705742341
Epoch 10, Loss: 0.6916302678959325
2345781 2345781
Epoch 1, Loss: 0.6915937741057614
Epoch 2, Loss: 0.6915629702516695
Epoch 3, Loss: 0.6915323496606389
Epoch 4, Loss: 0.6915005252554743
Epoch 5, Loss: 0.6914759125927827
Epoch 6, Loss: 0.691449770113317
Epoch 7, Loss: 0.6914301365898512
Epoch 8, Loss: 0.6914138218030514
E

In [18]:
board = random_board()

# Assuming you have the model architecture defined
model =  net(conv_size, conv_depth) # the model definition must be exactly the same as the saved model

# Load the model state
model.load_state_dict(torch.load('model_weights.pth'))


model.to(device)

# Don't forget to switch to eval mode if you're doing inference
model.eval()


print(model(torch.tensor(numpy.expand_dims(split_dims(board), 0), dtype=torch.float32).to(device)))
print(Da_fish(board, 10)/2000)
print(np.asarray((np.tanh(3*(Da_fish(board, 10)/abs(z).max()))+1)/2))
board

TypeError: ResNetChess.forward() takes 2 positional arguments but 3 were given

: 